# **재정정보 AI 검색 알고리즘 경진대회 - RAG, RAPTOR 적용**
> **RAG, RAPTOR 적용, No Finetuning**

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

---

💡 **NOTE**: 이 예제에서는 

1. LLM : [ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit](https://huggingface.co/sh2orc/Llama-3.1-Korean-8B-Instruct)
2. Embed Model : [intfloat/multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base)

---

# 1. 라이브러리 설치하기

In [ ]:

# 모델 가속화 및 메모리 관리
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes

# Transformer 기반 모델과 데이터셋 관련 라이브러리
%pip install transformers[torch] -U
%pip install datasets

# LangChain 관련 라이브러리
%pip install langchain
%pip install langchainhub
%pip install langchain_community
%pip install langchain-openai
%pip install langchain-anthropic
%pip install langchain-huggingface

# 문서 처리 및 문서 임베딩 라이브러리
%pip install PyMuPDF
%pip install sentence-transformers

# 벡터 데이터베이스 관련 라이브러리
%pip install faiss-cpu
# %pip install faiss-gpu
%pip install chromadb

# 데이터 시각화 및 분석 관련 라이브러리
%pip install umap-learn
%pip install scikit-learn
%pip install tiktoken

# 2. 라이브러리 불러오기

In [6]:
import os
import getpass
import unicodedata
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import umap

from typing import Dict, List, Optional, Tuple

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM,
)
from accelerate import Accelerator

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.storage import LocalFileStore

from sklearn.mixture import GaussianMixture

# 3. RAPTOR 트리 구축

## 3-1. UMAP 알고리즘을 사용하여 글로벌 임베딩 차원 축소 함수 구현

- 전역 클러스터링은 데이터 내의 전반적인 전체 패턴을 이해하기 위해 사용합니다. <br>
- 구현 함수 : `global_cluster_embeddings(embeddings, dim, n_neighbors, metric) -> np.ndarray`

<br>

> **UMAP (Univorm Manifold Approximation and Projection)**
- 고차원 데이터를 저차원으로 축소하는 비선형 기법입니다.
- 주로 시각화와 클러스터링 지원하는데 사용합니다.
- UMAP은 데이터 포인트 간의 유사성을 유지하며, 데이터를 낮은 차원으로 축소하여 중요한 패턴을 쉽게 분석할 수 있게 합니다.

In [10]:
RANDOM_SEED = 2024  # 재현성을 위한 고정된 시드 값

def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    UMAP을 사용하여 임베딩의 전역 차원 축소를 수행합니다.

    파라미터:
    - embeddings  : numpy 배열로 된 입력 임베딩.
    - dim         : 축소할 차원 값.
    - n_neighbors : 선택 사항; 각 점을 고려할 이웃의 수. (기본값 : 임베딩 수의 제곱근)
    - metric      : UMAP에 사용할 거리 측정 기준.

    반환값:
    - 지정된 차원으로 축소된 임베딩의 numpy 배열.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

## 3-2. UMAP 알고리즘을 사용하여 로컬 임베딩 차원 축소 함수 구현

- 지역 차원 축소는 특정 데이터 포인트와 그 주변 데이터들 간의 관계를 더 잘 반영하기 위해 사용합니다.
- 구현 함수 : `local_cluster_embeddings(embeddings, dim, num_neighbors, metric) -> np.ndarray`

In [11]:
def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    임베딩에 대해 지역 차원 축소를 수행합니다. 이는 일반적으로 전역 클러스터링 이후에 사용됩니다.

    파라미터:
    - embeddings    : numpy 배열로서의 입력 임베딩.
    - dim           : 축소할 차원 값.
    - num_neighbors : 각 점에 대해 고려할 이웃의 수.
    - metric        : UMAP에 사용할 거리 측정 기준.

    반환값:
    - 지정된 차원으로 축소된 임베딩의 numpy 배열.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

## 3-3. GMM과 BIC를 사용하여 최적의 클러스터 수 결정 함수 구현

- 주어진 임베딩 데이터를 기반으로 최적의 클러스터 수를 결정하는데 사용합니다.
- 즉, 클러스터링 문제에서 데이터를 가장 잘 설명하는 클러스터 수를 자동으로 찾기 위해 사용합니다.
- **가우시안 혼합 모델(Gaussian Mixture Model, GMM)** 과 **베이지안 정보 기준(BIC)** 를 사용하여 구현할 수 있습니다.
- 구현 함수 : `get_optimal_clusters(embeddings, max_clusters, random_state) -> int`

<br>

> **GMM (Gaussian Mixture Model, 가우시안 혼합 모델)**
- 다양한 클러스터에 걸쳐 데이터 포인트의 분포를 모델링합니다.
- 모델의 베이지안 정보 기준(BIC)를 평가하여 최적의 클러스터 수를 결정합니다.

> **BIC (Bayesian Information Criterion, 베이지안 정보 기준)**
- 모델의 적합도와 복잡도 간의 균형을 맞추는 지표로, 너무 많은 클러스터를 사용하여 과적합되는 것을 방지합니다.
- 따라서, 이 함수는 복잡성을 최소화하면서도 데이터를 잘 설명하는 클러스터 수를 찾는데 유용합니다.

In [12]:
def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    가우시안 혼합 모델(Gaussian Mixture Model)을 사용하여 베이지안 정보 기준(BIC)을 통해 최적의 클러스터 수를 결정합니다.

    파라미터:
    - embeddings: numpy 배열로서의 입력 임베딩.
    - max_clusters: 고려할 최대 클러스터 수.
    - random_state: 재현성을 위한 시드.

    반환값:
    - 발견된 최적의 클러스터 수를 나타내는 정수.
    """
    max_clusters = min(max_clusters, len(embeddings))  # 최대 클러스터 수와 임베딩의 길이 중 작은 값을 최대 클러스터 수로 설정
    n_clusters = np.arange(1, max_clusters)            # 1부터 최대 클러스터 수까지의 범위를 생성
    bics = []                                          # BIC 점수를 저장할 리스트
    for n in n_clusters: # 각 클러스터 수에 대해 반복
        gm = GaussianMixture(n_components=n, random_state=random_state) # 가우시안 혼합 모델 초기화
        gm.fit(embeddings)              # 임베딩에 대해 모델 학습
        bics.append(gm.bic(embeddings)) # 학습된 모델의 BIC 점수를 리스트에 추가
    return n_clusters[np.argmin(bics)]  # BIC 점수가 가장 낮은 클러스터 수를 반환

## 3-4. 주어진 임베딩 데이터에 대해 GMM을 사용하여 클러스터링을 수행하는 함수 구현

- 각 데이터 포인트(임베딩)가 특정 클러스터에 속할 확률을 계산하고, 확률 임계값을 사용하여 각 데이터 포인트를 클러스터에 할당합니다.
- 이 과정은 확률 임계값을 기반으로 수행됩니다.
- 최적의 클러스터 수를 결정하기 위해 이전에 구현해둔 `get_optimal_clusters`함수를 호출합니다.
- 결정된 클러스터 수를 바탕으로 가우시안 혼합 모델을 초기화하고, 입력된 임베딩에 대해 학습을 수행합니다.
- 각 임베딩에 대한 클러스터 할당 확률을 계산하고, 이 확률이 주어진 임계값을 초과하는 경우 해당 임베딩을 클러스터에 할당합니다.
- 함수는 최종적으로 임베딩의 클러스터 레이블과 결정된 클러스터 수를 튜플로 반환합니다.
- 구현 함수 : `GMM_cluster(embeddings, threshold, random_state) -> labels, n_clusters`

In [13]:
def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    확률 임계값을 기반으로 가우시안 혼합 모델(GMM)을 사용하여 임베딩을 클러스터링합니다.

    파라미터:
    - embeddings   : numpy 배열로서의 입력 임베딩.
    - threshold    : 임베딩을 클러스터에 할당하기 위한 확률 임계값.
    - random_state : 재현성을 위한 시드.

    반환값:
    - 클러스터 레이블과 결정된 클러스터 수를 포함하는 튜플.
    """
    n_clusters = get_optimal_clusters(embeddings)  # 최적의 클러스터 수를 구합니다.
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state) # 가우시안 혼합 모델을 초기화합니다.
    gm.fit(embeddings)                                         # 임베딩에 대해 모델을 학습합니다.
    probs = gm.predict_proba(embeddings)                       # 임베딩이 각 클러스터에 속할 확률을 예측합니다.
    labels = [np.where(prob > threshold)[0] for prob in probs] # 임계값을 초과하는 확률을 가진 클러스터를 레이블로 선택합니다.
    return labels, n_clusters                                  # 레이블과 클러스터 수를 반환합니다.

## 3-5. 고차원 임베딩 데이터를 다단계 클러스터링 수행 함수 구현

- 차원 축소, 글로벌 클러스터링, 각 글로벌 클러스터 내의 로컬 클러스터링을 순차적으로 실행하여 데이터를 보다 세분화된 방식으로 클러스터링 합니다.
- 이 과정은 고차원 데이터의 복잡한 구조를 보다 효과적으로 분석하고, 다양한 규모에서의 패턴을 포착하는데 도움을 줍니다.
- 구현 함수 : `perform_clustering(embeddings, dim, threshold) -> List[np.ndarray]`

> 동작 과정
1. 임베딩 크기 확인
    - 입력된 임베딩의 수가 `dim`+1 보다 작거나 같다면, 데이터가 충분하지 않다고 판단하고 기본적으로 각 임베딩을 하나의 클러스터로 할당합니다.
2. 글로벌 차원 축소
    - **UMAP** 을 사용하여 입력 임베딩의 차원을 `dim`차원으로 축소합니다. 이 과정을 통해 고차원 데이터를 분석하기 더 용이하게 만듭니다.
    - `global_cluster_embeddings` 함수를 호출하여 차원 축소를 수행합니다.
3. 글로벌 클러스터링
    - 축소된 임베딩에 대해 **GMM**을 사용하여 글로벌 클러스터링을 수행합니다.
    - `GMM_cluster` 함수를 호출하여 최적의 클러스터 수를 찾고, 각 임베딩이 속한 글로벌 클러스터를 결정합니다.
4. 로컬 클러스터링
    - 각 글로벌 클러스터에 속한 임베딩에 대해 추가적인 로컬 차원 축소 및 클러스터링을 수행합니다.
    - 글로벌 클러스터에 속한 임베딩들을 추출한 후, 다시 **UMAP**을 사용하여 차원 축소를 하고, **GMM**을 사용하여 `local_cluster_embeddings` 함수 호출을 통해 로컬 클러스터링을 수행합니다.
    - 각 로컬 클러스터에 속한 임베딩에 대해 클러스터 ID를 할당합니다.
5. 클러스터 ID 할당
    - 각 임베딩에 대해 할당된 글로벌 및 로컬 클러스터 ID를 결합하여 최종적으로 임베딩이 속하는 클러스터를 식별합니다.
    - 이 과정에서 글로벌 클러스터 내에서 로컬 클러스터링 결과가 최종 클러스터 ID로 포함됩니다.
6. 결과 반환
    - 각 임베딩이 속하는 최종 클러스터 ID 배열을 반환합니다. 이 배열은 모든 임베딩에 대해 글로벌 및 로컬 클러스터링 결과를 결합한 ID를 포함합니다.

In [14]:
def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    임베딩에 대해 차원 축소, 가우시안 혼합 모델을 사용한 클러스터링, 각 글로벌 클러스터 내에서의 로컬 클러스터링을 순서대로 수행합니다.

    파라미터:
    - embeddings : numpy 배열로 된 입력 임베딩.
    - dim        : UMAP 축소를 위한 목표 차원.
    - threshold  : GMM에서 임베딩을 클러스터에 할당하기 위한 확률 임계값.

    반환값:
    - 각 임베딩의 클러스터 ID를 포함하는 numpy 배열의 리스트.
    """
    if len(embeddings) <= dim + 1:
        # 데이터가 충분하지 않을 때 클러스터링을 피합니다.
        return [np.array([0]) for _ in range(len(embeddings))]

    # 글로벌 차원 축소
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # 글로벌 클러스터링
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # 각 글로벌 클러스터를 순회하며 로컬 클러스터링 수행
    for i in range(n_global_clusters):
        # 현재 글로벌 클러스터에 속하는 임베딩 추출
        global_cluster_embeddings_ = embeddings[
            np.array([i in gc for gc in global_clusters])
        ]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # 작은 클러스터는 직접 할당으로 처리
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # 로컬 차원 축소 및 클러스터링
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # 로컬 클러스터 ID 할당, 이미 처리된 총 클러스터 수를 조정
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters

## 3-6. 주어진 텍스트 문서에 대한 임베딩을 생성하는 함수 구현

- embd 객체를 사용하여 텍스트 데이터를 임베딩 벡터로 변환한 후, 클러스터링하고 원본 텍스트, 해당 임베딩, 그리고 할당된 클러스터 라벨을 포함하는 `pandas.DataFrame`을 반환합니다.
- 구현 함수 : `embed(texts) -> numpy.ndarray`

> 동작 과정
- 입력으로 텍스트 문서의 목록(`texts`)을 받습니다.
- `embd` 객체의 `embed_documents` 메소드를 사용하여 텍스트 문서의 임베딩을 생성합니다.
- 생성된 임베딩을 `numpy.ndarray` 형태로 변환하여 반환합니다.

In [15]:
def embed(texts):
    """
    텍스트 문서 목록에 대한 임베딩을 생성합니다.
    이 함수는 `embd` 객체가 존재한다고 가정하며, 이 객체는 텍스트 목록을 받아 그 임베딩을 반환하는 `embed_documents` 메소드를 가지고 있습니다.
    
    파라미터:
    - texts: List[str], 임베딩할 텍스트 문서의 목록입니다.
    
    반환값:
    - numpy.ndarray: 주어진 텍스트 문서들에 대한 임베딩 배열입니다.
    """
    text_embeddings = embd.embed_documents(
        texts
    )  # 텍스트 문서들의 임베딩을 생성합니다.
    text_embeddings_np = np.array(text_embeddings) # 임베딩을 numpy 배열로 변환합니다.
    return text_embeddings_np                      # 임베딩된 numpy 배열을 반환합니다.

## 3-7. 텍스트 데이터의 구조적 분석, 그룹화를 위한 통합 파이프라인 구현

- 주어진 텍스트 목록에 대해 임베딩을 생성하고, 생성된 임베딩을 기반으로 클러스터링을 수행하여 최종적으로 원본 텍스트, 해당 임베딩, 그리고 할당된 클러스터 라벨을 포함하는 **pandas.DataFrame**을 반환합니다.
- 구현 함수 : `embed_cluster_texts(texts) -> df`

> 동작 과정
1. 주어진 텍스트 목록에 대해 임베딩 생성 (임베딩 생성을 위해 `embed()`사용)
2. 생성된 임베딩을 기반으로 클러스터링 수행, 이 과정은 사전에 정의된 `perform_clustering()`함수를 사용
3. 결과를 저장하기 위해 **pandas.DataFrame**을 초기화
4. DataFrame에 원본 텍스트, 임베딩 리스트, 클러스터 라벨을 각각 저장

In [16]:
def embed_cluster_texts(texts):
    """
    텍스트 목록을 임베딩하고 클러스터링하여, 텍스트, 그들의 임베딩, 그리고 클러스터 라벨이 포함된 DataFrame을 반환합니다.
    이 함수는 임베딩 생성과 클러스터링을 단일 단계로 결합합니다. 임베딩에 대해 클러스터링을 수행하는 `perform_clustering` 함수의 사전 정의된 존재를 가정합니다.

    파라미터:
    - texts: List[str], 처리될 텍스트 문서의 목록입니다.

    반환값:
    - pandas.DataFrame: 원본 텍스트, 그들의 임베딩, 그리고 할당된 클러스터 라벨이 포함된 DataFrame입니다.
    """
    text_embeddings_np = embed(texts) # 임베딩 생성
    cluster_labels = perform_clustering(
        text_embeddings_np, 10, 0.1
    )  # 임베딩에 대해 클러스터링 수행
    df = pd.DataFrame()                   # 결과를 저장할 DataFrame 초기화
    df["text"] = texts                    # 원본 텍스트 저장
    df["embd"] = list(text_embeddings_np) # DataFrame에 리스트로 임베딩 저장
    df["cluster"] = cluster_labels        # 클러스터 라벨 저장
    return df

## 3-8. 텍스트 문서를 단일 문자열을 포맷팅하는 함수 구현

- **pandas**의 **DataFrame**에서 `text` 컬럼에 있는 여러 텍스트 문서를 하나의 단일 문자열로 포맷팅합니다.
- 각 텍스트 문서는 특정 구분자("--- --- \n --- --- ")로 연결되어 있으며, 최종적으로 연결된 하나의 큰 문자열을 반환합니다.
- 이 함수는 여러 텍스트 문서를 하나의 긴 문자열로 결합하기 위해 사용합니다.
- 구현 함수 : `fmt_txt(df) -> str`

In [17]:
def fmt_txt(df: pd.DataFrame) -> str:
    """
    DataFrame에 있는 텍스트 문서를 단일 문자열로 포맷합니다.

    파라미터:
    - df: 'text' 열에 포맷할 텍스트 문서가 포함된 DataFrame.

    반환값:
    - 모든 텍스트 문서가 특정 구분자로 결합된 단일 문자열.
    """
    unique_txt = df["text"].tolist()  # 'text' 열의 모든 텍스트를 리스트로 변환
    return "--- --- \n --- --- ".join(
        unique_txt
    )  # 텍스트 문서들을 특정 구분자로 결합하여 반환

## 3-9. 주어진 텍스트 목록에 대해 임베딩 생성, 클러스터링, 그리고 클러스터 내 텍스트 요약을 단계적으로 수행하는 파이프라인 구축

- 텍스트 데이터를 임베딩하고, 클러스터링하며, 각 클러스터에 대한 요약을 생성하는 과정을 수행합니다.
- 주어진 텍스트 목록에 대해 임베딩을 생성하고 유사성에 기반한 클러스터링을 진행합니다. 이 과정은 `df_clusters` 데이터프레임을 결과로 합니다. 이 데이터프레임에는 원본 텍스트, 임베딩, 그리고 클러스터 할당 정보가 포함됩니다.
- 클러스터 할당을 쉽게 처리하기 위해 데이터프레임 항목을 확장합니다. 각 행은 텍스트, 임베딩, 클러스터를 포함하는 새로운 데이터프레임으로 변환됩니다.
- 확장된 데이터프레임에서 고유한 클러스터 식별자를 추출하고, 각 클러스터에 대한 텍스트를 포맷팅하여 요약을 생성합니다. 이 요약은 `df_summary` 데이터프레임에 저장됩니다. 이 데이터프레임은 각 클러스터의 요약, 지정된 세부 수준, 그리고 클러스터 식별자를 포함합니다.
- 최종적으로, 함수는 두 개의 데이터프레임을 포함하는 튜플을 반환합니다. 첫 번째 데이터프레임은 원본 텍스트, 임베딩, 클러스터 할당 정보를 포함하며, 두 번째 데이터프레임은 각 클러스터에 대한 요약과 해당 세부 수준, 클러스터 식별자를 포함합니다.
- 구현 함수 : `embd_cluster_summarize_texts(texts: List[str], level: int) -> Tuple[pd.DataFrame, pd.DataFrame]`

In [18]:
def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    텍스트 목록에 대해 임베딩, 클러스터링 및 요약을 수행합니다. 이 함수는 먼저 텍스트에 대한 임베딩을 생성하고,
    유사성을 기반으로 클러스터링을 수행한 다음, 클러스터 할당을 확장하여 처리를 용이하게 하고 각 클러스터 내의 내용을 요약합니다.

    매개변수:
    - texts: 처리할 텍스트 문서 목록입니다.
    - level: 처리의 깊이나 세부 사항을 정의할 수 있는 정수 매개변수입니다.

    반환값:
    - 두 개의 데이터프레임을 포함하는 튜플:
      1. 첫 번째 데이터프레임(`df_clusters`)은 원본 텍스트, 그들의 임베딩, 그리고 클러스터 할당을 포함합니다.
      2. 두 번째 데이터프레임(`df_summary`)은 각 클러스터에 대한 요약, 지정된 세부 수준, 그리고 클러스터 식별자를 포함합니다.
    """

    # 텍스트를 임베딩하고 클러스터링하여 'text', 'embd', 'cluster' 열이 있는 데이터프레임을 생성합니다.
    df_clusters = embed_cluster_texts(texts)

    # 클러스터를 쉽게 조작하기 위해 데이터프레임을 확장할 준비를 합니다.
    expanded_list = []

    # 데이터프레임 항목을 문서-클러스터 쌍으로 확장하여 처리를 간단하게 합니다.
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # 확장된 목록에서 새 데이터프레임을 생성합니다.
    expanded_df = pd.DataFrame(expanded_list)

    # 처리를 위해 고유한 클러스터 식별자를 검색합니다.
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # 요약
    template = """여기 LangChain 표현 언어 문서의 하위 집합이 있습니다.
    
    LangChain 표현 언어는 LangChain에서 체인을 구성하는 방법을 제공합니다.
    
    제공된 문서의 자세한 요약을 제공하십시오.
    
    문서:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()

    # 각 클러스터 내의 텍스트를 요약을 위해 포맷팅합니다.
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # 요약, 해당 클러스터 및 레벨을 저장할 데이터프레임을 생성합니다.
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary

## 3-10 텍스트 데이터를 재귀적으로 임베딩, 클러스터링 및 요약하는 과정을 구현한 함수

- 주어진 텍스트 리스트를 임베딩, 클러스터링 및 요약하여 각 단계별로 결과를 저장합니다.
- 함수는 최대 지정된 재귀 레벨까지 실행되거나, 유일한 클러스터의 수가 1이 될 때까지 반복됩니다.
- 각 재귀 단계에서는 현재 레벨의 클러스터링 결과와 요약 결과를 데이터프레임 형태로 반환하고, 이를 결과 딕셔너리에 저장합니다.
- 만약 현재 레벨이 최대 재귀 레벨보다 작고, 유일한 클러스터의 수가 1보다 크다면, 현재 레벨의 요약 결과를 다음 레벨의 입력 텍스트로 사용하여 재귀적으로 함수를 호출합니다.
- 최종적으로 각 레벨별 클러스터 데이터프레임과 요약 데이터프레임을 포함하는 딕셔너리를 반환합니다.

In [19]:
def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    지정된 레벨까지 또는 고유 클러스터의 수가 1이 될 때까지 텍스트를 재귀적으로 임베딩, 클러스터링, 요약하여
    각 레벨에서의 결과를 저장합니다.

    매개변수:
    - texts: List[str], 처리할 텍스트들.
    - level: int, 현재 재귀 레벨 (1에서 시작).
    - n_levels: int, 재귀의 최대 깊이.

    반환값:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], 재귀 레벨을 키로 하고 해당 레벨에서의 클러스터 DataFrame과 요약 DataFrame을 포함하는 튜플을 값으로 하는 사전.
    """
    results = {}  # 각 레벨에서의 결과를 저장할 사전

    # 현재 레벨에 대해 임베딩, 클러스터링, 요약 수행
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # 현재 레벨의 결과 저장
    results[level] = (df_clusters, df_summary)

    # 추가 재귀가 가능하고 의미가 있는지 결정
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # 다음 레벨의 재귀 입력 텍스트로 요약 사용
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # 다음 레벨의 결과를 현재 결과 사전에 병합
        results.update(next_level_results)

    return results

# 4. 모델 초기화

임베딩 모델은 `intfloat/multilingual-e5-base`을 허깅페이스에서 가져와서 사용합니다. <br>
**Cached Embedding** 을 사용하여 한번 계산된 임베딩을 저장해 두었다가, 동일한 텍스트에 대해 이전에 계산된 임베딩 값을 계속 활용하여 중복 계산을 피합니다. 이로 인해 리소스을 좀 더 효율적으로 사용할 수 있어 연산 비용을 절감할 수 있습니다.

In [12]:
def setup_embeddings(model_path="intfloat/multilingual-e5-base"):
    """임베딩 모델 설정 및 캐시 사용"""
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embd = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    
    store = LocalFileStore("./cache/")
    
    # Cache Embedding 사용
    cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
        underlying_embeddings=embd, 
        document_embedding_cache=store, 
        namespace=model_path
    )
    
    return cached_embeddings

사전에 `sh2orc/Llama-3.1-Korean-8B-Instruct`(Llama3 모델 한국어 파인튜닝 모델)을 대상으로 GPTQ 양자화 시켜준 모델을 허깅페이스에서 가져옵니다.

In [8]:
def setup_llm_pipeline():
    # 모델 ID (Hugging Face Hub에서 가져온 양자화된 모델 ID)
    model_id = "ormor/Llama-3.1-Korean-8B-Instruct-GPTQ-4bit"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True
    )
    
    print(f"#### [ model ] ####\n{model}\n###################")

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=450,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf

# 5. 데이터셋 로드 및 벡터 DB 생성

In [10]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 파일로드, 텍스트 추출"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    doc_texts = []
    
    # 모든 페이지의 텍스트 추출
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        doc_texts.append(Document(page_content=text, metadata={"page": page_num}))

    return doc_texts    


def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """
    데이터프레임에서 PDF 처리, 
    여러 PDF 파일을 처리하고, 각 파일에 대해 벡터 DB와 리트리버를 생성하는 함수
    딕셔너리에 pdf명을 키로해서 DB, retriever 저장
    """
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 
        docs_texts = process_pdf(full_path)
        
        # 트리 구축
        leaf_texts = docs_texts  # 문서 텍스트를 리프 텍스트로 설정
        results = recursive_embed_cluster_summarize(
            leaf_texts, level=1, n_levels=3
        )  # 재귀적으로 임베딩, 클러스터링 및 요약을 수행하여 결과를 얻음
        
        # leaf_texts를 복사하여 all_texts를 초기화
        all_texts = leaf_texts.copy()
        
        # 각 레벨의 요약을 추출하여 all_texts에 추가
        for level in sorted(results.keys()):
            summaries = results[level][1]["summaries"].tolist()  # 현재 레벨의 DataFrame에서 요약을 추출합니다.
            all_texts.extend(summaries) # 현재 레벨의 요약을 all_texts에 추가합니다.
        
        # FAISS 벡터 DB 생성
        embeddings = setup_embeddings()
        vectorstore = FAISS.from_texts(texts=all_texts, embedding=embeddings)
        
        # 로컬에 FAISS DB 인덱스가 이미 존재하는지 확인하고, 있다면 병합
        DB_INDEX = f"RAPTOR_{pdf_title}"
        if os.path.exists(DB_INDEX):
            local_index = FAISS.load_local(DB_INDEX, embeddings)
            local_index.merge_from(vectorstore)
            local_index.save_local(DB_INDEX)
        else:
            vectorstore.save_local(folder_path=DB_INDEX)
        
        # Retriever 생성
        retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
            'db': vectorstore,
            'retriever': retriever
        }
    return pdf_databases

# 6. LLM 추론, 답변 생성

In [21]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    # 문서의 페이지 내용을 이어붙여 반환합니다.
    return "\n\n".join(doc.page_content for doc in docs)

def generate_answers(df, pdf_databases, llm):
    """DataFrame의 각 질문에 대해 답변을 생성"""
    results = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
        source = normalize_string(row['Source'])
        question = row['Question']

        # 정규화된 키로 데이터베이스 검색
        normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
        retriever = normalized_keys[source]['retriever']

        # # RAG 체인 구성
        # template = """
        # 다음 정보를 바탕으로 질문에 답하세요:
        # {context}

        # 질문: {question}
        
        # 주어진 질문에만 답변하세요. 문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요.
        # 답변:
        # """
        # prompt = PromptTemplate.from_template(template)
        # 프롬프트 생성
        prompt = hub.pull("rlm/rag-prompt")
        
        rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        # 답변 추론
        full_response = rag_chain.invoke(question)

        # 결과 저장
        results.append({
            "Source": row['Source'],
            "Source_path": row['Source_path'],
            "Question": question,
            "Answer": full_response
        })

    return results

---

# 7. 최종 통합 및 실행

## 7-1. Vector DB 생성

In [1]:
# google drive
# !ls /content/drive/MyDrive/Contest/Dacon_Financial_Search/open/

# local
!ls /Users/ormor/root/contest/dacon_financial_search/open

sample_submission.csv test_source           train_source
test.csv              train.csv


In [ ]:
# 임베딩 모델 설정
embeddings = setup_embeddings()

In [ ]:
# PDF 파일들 처리 및 벡터 DB 생성

# google drive
# base_directory = r'/content/drive/MyDrive/Contest/Dacon_Financial_Search/open' # pdf source 경로
# df = pd.read_csv(r'/content/drive/MyDrive/Contest/Dacon_Financial_Search/open/test.csv') # csv 파일 경로

# local
base_directory = f'/Users/ormor/root/contest/dacon_financial_search/open' # pdf source 경로
df = pd.read_csv(f'{base_directory}/test.csv') # csv 파일 경로
pdf_databases = process_pdfs_from_dataframe(df, base_directory, embeddings)

## 7-2. 추론 실행, 답변 생성

In [6]:
# Langsmith 설정
# 환경 변수 설정, 콜솔에 Langchain Api Key 입력
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass(prompt="Enter your LangChain API key: ")

# 언어 모델 초기화
llm = setup_llm_pipeline()

# 질문에 대한 답변 생성
results = generate_answers(df, pdf_databases, llm) # list 반환

# 8. 제출용 파일 생성 및 저장

In [ ]:
# 제출 샘플 파일 로드
submit_df = pd.read_csv("./sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘") # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./baseline_submission.csv", encoding='UTF-8-sig', index=False)